In [33]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import sklearn as skl
import nltk
import csv
import networkx as nx
import operator
import matplotlib.pyplot as plt
import pandas as pd
import re
from collections import defaultdict

In [34]:
node_inf_raw = pd.read_csv("./node_information.csv")
node_inf = node_inf_raw.values
all_auth = np.array([(re.sub(r',Jr.?', 'Jr',
                            re.sub(r',(?=[a-z])', '',
                                   re.sub(r'\([^)]*\)?', '',
                                          re.sub(r' ', '' , a[3])))).split(","),a[0]) if type(a[3]) == str else [] for a in node_inf])
print(all_auth[:100])


[(['PaulS.Aspinwall'], 1001L) (['M.Cvetic', 'H.Lu', 'C.N.Pope'], 1002L)
 (['Y.S.Myung', 'GungwonKang'], 1003L) (['AdamD.Helfer'], 1004L)
 (['J.Fuchs', 'C.Schweigert'], 1005L) (['RudolfHaag'], 1006L)
 (['ShengLi', 'YongZhang', 'ZhongyuanZhu'], 1007L)
 (['DonaldSpector'], 1008L) (['GourangaC.Nayak', 'WalterGreiner'], 1009L)
 (['A.V.Belitsky', 'S.Voren', 'P.vanNieuwenhuizen'], 1010L)
 (['TsunehideKuroki'], 1011L)
 (['V.Fateev', 'A.Zamolodchikov', 'Al.Zamolodchikov'], 1012L)
 (['RainerDick', 'DzoMikulovicz'], 1013L) (['Franz-MarcBoas'], 1014L)
 (['AliceRogers'], 1015L) (['EnriqueAlvarez', 'CesarGomez'], 1016L)
 (['R.S.Ward'], 1017L) list([])
 (['I.Klich', 'J.Feinberg', 'A.Mann', 'M.Revzen'], 1019L)
 (['STTsou'], 1020L)
 (['ZalanHorvath', 'RobertL.Karp', 'LaszloPalla'], 1021L)
 (['BogdanDamski'], 1022L) list([])
 (['JeromeP.Gauntlett', 'GaryW.Gibbons', 'ChristopherM.Hull', 'PaulK.'], 1024L)
 (['BrianR.Greene', 'C.I.Lazaroiu'], 1025L)
 (['AllenC.Hirshfeld', 'ThomasSchwarzweller'], 1026L)
 ([

In [35]:
authors = defaultdict(list)
for a,c in zip(all_auth,node_inf):
    if a != []:
        #print(a)
        (b,i) = a
        for auteur in b:
            authors[auteur].append(i)

authors[''] = None

print(authors)

defaultdict(<type 'list'>, {'': None, 'H.Samtleben': [6034L, 10076L, 103032L, 106153L, 112035L, 207206L, 9710210L, 9804152L, 9903111L, 9912111L], 'DanijelJurman': [304006L], 'Rui-hongYue': [109091L], 'ChristianR\\"omelsberger': [2037L, 3263L], 'RaikoP.Zaikov': [9303087L, 9304075L, 9308016L, 9308017L], 'S.A.Antonenko': [9803264L], 'A.Sevrin': [10151L, 11018L, 11264L, 101018L, 105274L, 9209037L, 9303051L, 9303058L, 9306033L, 9306059L, 9306135L, 9403183L, 9408033L, 9412198L, 9706218L, 9801080L, 9812207L, 9905141L], 'TiloWettig': [9605110L, 9704055L, 9811044L], 'V.Bazhanov': [9412229L, 9812091L, 9812247L], 'A.Gerasimov': [5053L, 9405011L, 9601161L], 'A.Minzoni': [9606092L], 'AlexerYu.Kamenshchik': [6220L, 6221L, 10288L, 9601006L, 9603021L, 9604182L, 9604194L, 9606132L, 9609178L, 9610075L, 9707138L, 9707168L, 9802059L, 9803036L], 'AndreiBytsenko': [9303061L, 9405090L], 'P.Windey': [9704190L], 'SujayK.Ashok': [211291L], 'Sergio.A.Pernice': [9612125L], 'V.Frolov': [12048L, 12252L, 12260L, 212

In [36]:
nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kingr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
print(node_inf[:10][1])

[1002L 2000L 'domain walls and massive gauged supergravity potentials'
 'M. Cvetic, H. Lu, C.N. Pope' 'Class.Quant.Grav.'
 'we point out that massive gauged supergravity potentials for example those arising due to the massive breathing mode of sphere reductions in m-theory or string theory allow for supersymmetric static domain wall solutions which are a hybrid of a randall-sundrum domain wall on one side and a dilatonic domain wall with a run-away dilaton on the other side on the anti-de sitter ads side these walls have a repulsive gravity with an asymptotic region corresponding to the cauchy horizon while on the other side the runaway dilaton approaches the weak coupling regime and a non-singular attractive gravity with the asymptotic region corresponding to the boundary of spacetime we contrast these results with the situation for gauged supergravity potentials for massless scalar modes whose supersymmetric ads extrema are generically maxima and there the asymptotic regime transvers

In [ ]:
def graph_articles(citation_set, node_inf, directed_or_not = 'n'):
    if directed_or_not == 'y':
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    for i in citation_set:
        if i[2] == '1':
            y1 = 0
            y2 = 0
            for node in node_info:
                if node[0] == int(i[0]):
                    y1 = node[1]
                if node[0] == int(i[1]):
                    y2 = node[1]
            if y1 < y2: 
                G.add_edge(i[0], i[1])
            else:
                G.add_edge(i[1], i[0])
        else:
            G.add_node(i[0])
            G.add_node(i[1])
    return G

In [ ]:
G = graph_articles(training_set, node_info, directed_or_not = 'y')

In [ ]:
authors_h = dict()
for aut in authors.keys():
    liste = [G.node[i].in_degree() for i in authors[aut]]
    listes = liste.sort()
    i=1
    j=0
    while listes[j] >= i:
        i+=1
        j+=1
    authors_h[aut] = i

print(authors_h)
        
#Le dictionnaire qui renvoie a chaque auteur son h-index
#Il ne reste plus qu'à associer à chaque node la moyenne de l'h-index de ses auteurs
#La feature sera ensuite calculée comme le page rank/club: addition des h_index des 2 nodes

In [ ]:
for i in G:
    hlist = [authors_h[j] for j in authors[i]]
    G.node[i]['h-index'] = sum(hlist)/len(hlist)

In [ ]:
h-index-features = []
for citation in training_set:
        h-index-features.append(G.node[citation[0]]['h-index'] + G.node[citation[1]]['h-index'])

In [ ]:
print(h-index-features)